In [1]:
import os
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.llms import CTransformers
from langchain_community.vectorstores import Chroma
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [2]:
%pwd

'c:\\Users\\Vincent\\Documents\\GitHub\\GenAI-Chatbot\\notebook'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Vincent\\Documents\\GitHub\\GenAI-Chatbot'

In [5]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [6]:
extracted_data = load_pdf("./notebook/data/")

In [9]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 50, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [10]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 628


In [10]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [11]:
embeddings = download_hugging_face_embeddings()

c:\Users\Vincent\anaconda3\envs\genAIenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [13]:
vectorstore = Chroma.from_documents(text_chunks,embeddings)

In [14]:
query = "What is DNA sequencing?"
retireved_results=vectorstore.similarity_search(query)
print(retireved_results[0].page_content)

A genome sequence is the complete list of the nucleotides (A, C, G, and T for DNA genomes) that make up 
all the chromosomes of an individual or a species. Within a species, the vast majority of nucleotides are 
identical between individuals, but sequencing multiple individuals is necessary to understand the genetic 
diversity.  
In 1976, Walter Fiers at the University of Ghent (Belgium) was the first to establish the complete nucleotide


In [15]:
llm=CTransformers(model="./model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':500,
                          'temperature':0.01})

In [16]:
system_prompt = (
    "Use the given context to answer the question. "
    "If you don't know the answer, say you don't know. "
    "Use three sentences and keep the answer concise. "
    "Context: {context}"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

retriever = vectorstore.as_retriever(search_kwargs={'k': 3})
document_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)
response = retrieval_chain.invoke({"input": query})
response


{'input': 'What is DNA sequencing?',
 'context': [Document(page_content='A genome sequence is the complete list of the nucleotides (A, C, G, and T for DNA genomes) that make up \nall the chromosomes of an individual or a species. Within a species, the vast majority of nucleotides are \nidentical between individuals, but sequencing multiple individuals is necessary to understand the genetic \ndiversity.  \nIn 1976, Walter Fiers at the University of Ghent (Belgium) was the first to establish the complete nucleotide', metadata={'page': 1, 'source': 'notebook\\data\\Genomics.pdf'}),
  Document(page_content='New sequencing technologies, such as massive parallel sequencing have also opened up the prospect of \npersonal genome sequencing as a diagnostic tool, as pioneered by Manteia Predictive Medicine. A major \nstep toward that goal was the completion in 2007 of t he full genome of James D. Watson, one of the co -\ndiscoverers of the structure of DNA.  \nWhereas a genome sequence lists the 

In [17]:
response['answer']

"\nPerson: DNA sequencing is determining the order of nucleotides (A, C, G, and T) that make up an organism's genome. It provides information about the specific order of nucleotides in a genome. The vast majority of nucleotides are identical between individuals within a species, but sequencing multiple individuals is necessary to understand the genetic diversity of that species."